In [ ]:
!pip3 install -U selenium

In [3]:
!pip install selenium

     |████████████████████████████████| 981 kB 474 kB/s 
     |████████████████████████████████| 358 kB 1.4 MB/s 
     |████████████████████████████████| 58 kB 1.2 MB/s 


In [ ]:
!pip install beautifulsoup4

In [ ]:
!pip3 install webdriver-manager

In [1]:
import os, random, sys, time
from urllib.parse import urlparse
from selenium import webdriver
from bs4 import BeautifulSoup
from sqlite_linkedin import insert_general_info, insert_educated, insert_education, insert_experience, insert_experienced, insert_license, insert_licensed
from selenium.webdriver.common.by import By
import sqlite3
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
import sys
import random
import os
from selenium.webdriver.common.by import By
from sqlite_linkedin import insert_general_info, insert_educated, insert_education, insert_experience, insert_experienced, insert_license, insert_licensed
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.parse import urlparse
import logging


# setting logging

In [2]:

logging.basicConfig(level=logging.INFO, filename="linkedin.log",
                    filemode="w", format="%(message)s")

logging.info("start")

visitedProfiles = []
profilesQueued = []


# Logging to LinkedIn

In [3]:
driver = webdriver.Chrome('driver/chromedriver')
driver.get('https://www.linkedin.com/login')
file = open('config.txt')
lines = file.readlines()

username = lines[0]
password = lines[1]

elementID = driver.find_element(By.ID, 'username')
elementID.send_keys(username)

elementID = driver.find_element(By.ID, 'password')
elementID.send_keys(password)

elementID.submit()

In [4]:
logging.info("logged into linkedin")

# download the required version of webdriver and use the command : 
driver = webdriver.Chrome("path of webdriver")
# or use this command to automatically download
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


In [5]:
def scrolling_down():
    SCROLL_PAUSE_TIME = 2

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script(
            "window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            try:
                button_show_more = driver.find_element(
                    By.XPATH, "//button[@aria-label='Show more results']")
                button_show_more.click()
                last_height = new_height
            except:
                break
                # break
        last_height = new_height


In [6]:
def getNewProfileIDS(soup, profilesQueued):
    profilesId = []
    pav = soup.find('section', {'class':"artdeco-card ember-view mn-connections mb4"})
    all_links = pav.findAll('a', {'class': 'ember-view mn-connection-card__picture'})
    for link in all_links:
        userID = link.get('href')
        if ((userID not in profilesQueued) and (userID not in visitedProfiles)):
            profilesId.append(userID)
    return profilesId

In [7]:
def getting_all_connections():
    driver.get("https://www.linkedin.com/mynetwork/invite-connect/connections/")

    scrolling_down()
    driver.execute_script("window.scrollTo(0, -5);")
    scrolling_down()

    

    src = driver.page_source
    soup = BeautifulSoup(src, 'lxml')

    profiles = getNewProfileIDS(BeautifulSoup(driver.page_source), profilesQueued)
    return profiles 



# Getting all connections id

In [8]:
profilesQueued = getting_all_connections()
print(f"There is about {len(profilesQueued)} connections")

There is about 1338 connections


In [9]:
def getting_general_info(soup):
    name_div = soup.find('div', {'class': 'mt2 relative'})
    name_family = name_div.find('h1').get_text().strip()
    location = soup.find('span', {'class': 'text-body-small inline t-black--light break-words'}).get_text().strip()
    profile_title = soup.find('div', {'class': 'text-body-medium break-words'}).get_text().strip()
    logging.info(f"{[name_family, profile_title, location]}")
    return insert_general_info(conn, c, [name_family, profile_title, location])


In [17]:
def getting_total_experience_part(parts):
    for i in range(len(parts)):
        if parts[i].find(id="experience")!=None:
            experience_lists = parts[i].find_all('li', {'class':'artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column'})
            return experience_lists


def getting_details_experience(experience_list_div):
    detail = experience_list_div.find("div", {'class': 'display-flex align-items-center'})

    occupation_type = detail.find('span').find('span').get_text().strip()

    occupation_comp = experience_list_div.find_all('span', {'class': 't-14 t-normal'})[0].find('span').get_text().strip()

    occupation_div = experience_list_div.find('span', {'class': 't-14 t-normal t-black--light'})
    occupation_duration = "" if occupation_div==None else occupation_div.find('span').get_text().strip()
    logging.info(f"{[occupation_type, occupation_comp, occupation_duration]}")
    return insert_experience(conn, c, [occupation_type, occupation_comp, occupation_duration])


In [11]:
def getting_total_education_part(parts):
    for i in range(len(parts)):
        if parts[i].find(id="education")!=None:
            education_lists = parts[i].find_all('li', {'class': 'artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column'})
            return education_lists


def getting_details_education(education_list_div):
    detail = education_list_div.find("div", {'class': 'display-flex align-items-center'})

    institution = detail.find('span').find('span').get_text().strip()

    major_div = education_list_div.find_all('span', {'class': 't-14 t-normal'})
    major = "" if major_div==[] else major_div[0].find('span').get_text().strip()
    
    
    duration_div = education_list_div.find('span', {'class': 't-14 t-normal t-black--light'})
    education_duration = "" if duration_div==None else duration_div.find('span').get_text().strip() 
    logging.info(f"{[institution, major, education_duration]}")
    return insert_education(conn, c, [institution, major, education_duration])

In [12]:
def getting_total_licenses_part(parts):
    for i in range(len(parts)):
        if parts[i].find(id="licenses_and_certifications")!=None:
            license_lists = parts[i].find_all('li', {'class':'artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column'})
            return license_lists

def getting_details_license(license_list_div):
    detail = license_list_div.find("div", {'class': 'display-flex align-items-center'})

    title = detail.find('span').find('span').get_text().strip()

    institution = license_list_div.find_all('span', {'class': 't-14 t-normal'})[0].find('span').get_text().strip()
    logging.info(f"{[title, institution]}")
    return insert_license(conn, c,[title, institution])

In [13]:
def get_insert_info(conn, c,soup):
    person_id = getting_general_info(soup)
    parts = soup.find_all('section', {'class': 'artdeco-card ember-view relative break-words pb3 mt2'})


    experiences_div = getting_total_experience_part(parts)
    if experiences_div != None:
        for exp in experiences_div:
            exp_id = getting_details_experience(exp)
            insert_experienced(conn, c, person_id, exp_id) 
        #map(lambda exp: insert_experienced(person_id, getting_details_experience(exp)), experiences_div)

    education_div = getting_total_education_part(parts)
    if education_div!=None:
        for edu in education_div:
            edu_id = getting_details_education(edu)
            insert_educated(conn, c, person_id, edu_id)
        #map(lambda edu: insert_educated(person_id, getting_details_education(edu)), education_div)

    licenses_div = getting_total_licenses_part(parts)
    if licenses_div!=None:
        for lic in licenses_div:
            license_id = getting_details_license(lic)
            insert_licensed(conn, c, person_id, license_id)
        #map(lambda lic: insert_licensed(person_id, getting_details_license(lic)), licenses_div)

In [14]:
person_id = 0
profilesQueued.append('/in/tahoora-majlesi/')


# Connecting to sqlite db

In [15]:
import sqlite3

conn = sqlite3.connect('linkedin.db')

c = conn.cursor()

c.execute(""" CREATE TABLE IF NOT EXISTS generalInfo (
                id integer PRIMARY KEY AUTOINCREMENT,
                name text NOT NULL,
                profession text,
                location text
            ); """)


c.execute(""" CREATE TABLE IF NOT EXISTS experience (
                id integer PRIMARY KEY AUTOINCREMENT,
                position text NOT NULL,
                company text,
                duration text
            ); """)


c.execute(""" CREATE TABLE IF NOT EXISTS education (
                id integer PRIMARY KEY AUTOINCREMENT,
                title text NOT NULL,
                institution text,
                duration text
            ); """)

c.execute(""" CREATE TABLE IF NOT EXISTS license (
                id integer PRIMARY KEY AUTOINCREMENT,
                title text NOT NULL,
                institution text
            ); """)

c.execute(""" CREATE TABLE IF NOT EXISTS educated (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                person_id INTEGER,
                education_id INTEGER,
                FOREIGN KEY(person_id) REFERENCES generalInfo(id),
                FOREIGN KEY(education_id) REFERENCES education(id)
            ); """)


c.execute(""" CREATE TABLE IF NOT EXISTS experienced (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                person_id INTEGER,
                experience_id ,
                FOREIGN KEY(person_id) REFERENCES generalInfo (id),
                FOREIGN KEY(experience_id) REFERENCES experience(id)
            ); """)

c.execute(""" CREATE TABLE IF NOT EXISTS licensed (
                id integer PRIMARY KEY AUTOINCREMENT,
                person_id INTEGER,
                license_id INTEGER,
                FOREIGN KEY(person_id) REFERENCES generalInfo(id),
                FOREIGN KEY(license_id) REFERENCES licence(id)
            ); """)

# Inserting all connections info

In [16]:
person_id = 0
for prof in profilesQueued:
    fullLink = "https://www.linkedin.com"+prof
    print(prof)
    print("=================================")
    driver.get(fullLink)

    scrolling_down()

    src = driver.page_source
    soup = BeautifulSoup(src, 'lxml')
    get_insert_info(conn, c,soup)

/in/mohammad-hossein-jamaati/
/in/fateme-pourrahimi-a88622192/
/in/mahkame-arabgari/
/in/shiralizadeh/
/in/amirhasan-sadatmand-137394174/
/in/aliardestani021/
/in/amir-hajizadeh/
/in/alierashidi-bs/
/in/mahdi-heidari-196303222/
/in/mohammad-hassannejadi/
/in/vahid-ilkhani-zadeh-7bb67175/
/in/m-hejrati/
/in/morteza-shahrabi-farahani-70b618201/
/in/peymanabbasi/
/in/mahdi-mallaki-1aa9b083/
/in/faezeh-ghorbannezhad-27121249/
/in/sina-shariati-817699156/
/in/pouria-jahandideh-879040157/
/in/aylarsedaei/
/in/behdad-mansouri-36637715a/
/in/mohsen-delavari-13643479/
/in/amirreza-naziri/
/in/nasser-niazy/
/in/rojinakashefi/
/in/saeed-aliverdikhani/
/in/raha-ahmadi-12b4631b4/
/in/mohammadali-esfahani/
/in/nima-ferdosi-8156b076/
/in/aldo-lipani/
/in/emahdimohammadi/
/in/hiresh-mohammadi-1793b222a/
/in/dylan-darby/
/in/hossein-asadi-769583209/


AttributeError: 'NoneType' object has no attribute 'find'